# Inspecting Satellite Imagery using Rasterio
## A first look at analyzing satellite data with Python

At this point, you've explored different ways of searching for, filtering, and downloading satellite imagery. Now let's use one of these acquired datasets and dig into it a bit with Python.

Here we're going to use a Python library called `rasterio`: you may be familiar with it already, or perhaps with the related C library, `GDAL`.

In [50]:
import rasterio
satdat = rasterio.open("example.tif")

## Basic details
What can we learn about this satellite image using just Python?

In [51]:
# Get the bounding box of this GeoTIFF
satdat.bounds

BoundingBox(left=540759.0, bottom=3754401.0, right=568047.0, top=3767985.0)

In [63]:
# Get dimensions, in map units (here, that's meters)

width = satdat.bounds.right - satdat.bounds.left
height = satdat.bounds.top - satdat.bounds.bottom

print("Width: {}, Height: {}".format(width, height))

Width: 27288.0, Height: 13584.0


In [64]:
# Get dimensions, in pixels
px_width = satdat.width
px_height = satdat.height
print("Width: {}, Height: {}".format(px_width, px_height))

Width: 9096, Height: 4528


In [65]:
# How many meters to a pixel?

w = width / px_width
h = height / px_height

w, h

(3.0, 3.0)

In [66]:
# Get coordinate reference system
satdat.crs

CRS({'init': 'epsg:32611'})

In [59]:
# Get coordinates of top-left & bottom right corners
# NOTE: how to do this depends on your Rasterio version:
# the below example may generate a FutureWarning, which is safe to ignore here

try:
    topleft = satdat.transform * (0, 0)
    botright = satdat.transform * (width, height)
    
except TypeError:
    topleft = satdat.affine * (0, 0)
    botright = satdat.affine * (width, height)
    
print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (540759.0, 3767985.0)
Bottom right corner coordinates: (568047.0, 3754401.0)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Bands
So far, we haven't done too much raster-specific work yet. Since we know we're inspecting a multispectral satellite image, let's see what we can learn about its bands.

In [ ]:
# Get the number of bands by listing their indices
satdat.indexes

Because we know we're look at a PlanetScope 4-band analytic satellite image, we can define the bands by their order:

In [ ]:
# PlanetScope 4-band band order: BGRN

blue = satdat.read(1)
green = satdat.read(2)
red = satdat.read(3)
nir = satdat.read(4)

In [ ]:
# bands are stored as numpy arrays

type(blue)

In [ ]:
blue

In [ ]:
# Output min & max pixel values in each band

print(blue.min(), blue.max())
print(green.min(), green.max())
print(red.min(), red.max())
print(nir.min(), nir.max())

## Pixels

In [61]:
# Let's grab the pixel 5km east and 5km south of the upper left corner

# get the pixel 
px_x = satdat.bounds.left + 5000
px_y = satdat.bounds.top - 5000

row, col = satdat.index(px_x, px_y)

# Now let's look at the value of Band1 ("blue") at this pixel
blue[row, col]

6382